In [1]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime
import copy

In [2]:
async def fetch_data_binance(session, token, start_time, end_time):
    url = 'https://fapi.binance.com/fapi/v1/klines'
    params = {
        'symbol': token,
        'interval': '1m',
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000
    }
    async with session.get(url, params = params) as response:
        return await response.json()

async def process_token_binance(session, token, start_time, end_time):
    time_intervals = [start_time]
    cur_time = start_time
    while cur_time < end_time:
        time_intervals.append(cur_time + 1000 * 60 * 1000)
        cur_time = time_intervals[-1]
    
    frames = []
    for i in range(len(time_intervals) - 1):
        result = await fetch_data_binance(session, token, time_intervals[i], time_intervals[i+1])
        try:
            cur_df = pd.DataFrame(result).iloc[:, :6]
            cur_df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
            frames.append(cur_df)
        except ValueError:
            print(f"Ticker {token} was skipped: {result['msg']} \n")
            return None, None
            
    
    all_frame = pd.concat(frames, ignore_index = True)
    all_frame.sort_values(by = 'time', inplace = True)
    all_frame.reset_index(drop = True, inplace = True)
    all_frame[['open', 'high', 'low', 'close', 'volume']] = all_frame[['open', 'high', 'low', 'close', 'volume']].astype('float')
    all_frame['time'] = all_frame['time'].astype('str')
    all_frame['token'] = token
    
    result_frame = all_frame
    condition1 = (result_frame.high / result_frame.low)
    condition1 = list(condition1[condition1 >= 1.015].index)
    
    condition2 = abs((result_frame.open / result_frame.close) - 1)
    condition2 = list(condition2[condition2 < 0.007].index)
    
    condition3 = abs((result_frame.close / result_frame.low))
    condition3 = list(condition3[condition3 >= 1.01].index)

    condition4 = abs((result_frame.high / result_frame.open))
    condition4 = list(condition4[condition4 >= 1.01].index)
    
    cond1235 = set.intersection(set(condition1), set(condition2), set(condition3))
    cond1245 = set.intersection(set(condition1), set(condition2), set(condition4))

    check_frame = copy.deepcopy(result_frame.loc[sorted(list(cond1235) + list(cond1245))])
    check_frame['normal_time'] = [datetime.fromtimestamp(int(t) / 1000.0) for t in check_frame.time]

    if len(check_frame) > 0:
        return token, check_frame
    return None, None

async def get_token_outliear_binance(tokens, start_time = 1722978000000, end_time = 1723486810000):
    check_tokens = []
    return_dict = {}

    async with aiohttp.ClientSession() as session:
        tasks = [process_token_binance(session, token, start_time, end_time) for token in tokens]
        results = await asyncio.gather(*tasks)

    for token, check_frame in results:
        if token:
            check_tokens.append(token)
            return_dict[token] = check_frame

    return return_dict

In [3]:
return_dict = await get_token_outliear_binance(tokens = ['ROSEUSDT', 'SAGAUSDT'])

In [4]:
return_dict.keys()

dict_keys(['ROSEUSDT', 'SAGAUSDT'])

In [5]:
return_dict['ROSEUSDT']

,time,open,high,low,close,volume,token,normal_time
2682,1723138920000,0.06299,0.06401,0.06293,0.06318,10535364.0,ROSEUSDT,2024-08-08 19:42:00


In [6]:
return_dict['SAGAUSDT']

,time,open,high,low,close,volume,token,normal_time
7395,1723421700000,1.2299,1.2502,1.2176,1.2344,1253406.7,SAGAUSDT,2024-08-12 02:15:00
7395,1723421700000,1.2299,1.2502,1.2176,1.2344,1253406.7,SAGAUSDT,2024-08-12 02:15:00
8257,1723473420000,1.6147,1.6413,1.6111,1.6241,2275311.1,SAGAUSDT,2024-08-12 16:37:00
8381,1723480860000,1.7068,1.7300,1.6992,1.7029,2571395.4,SAGAUSDT,2024-08-12 18:41:00
